In [64]:
#6.3.1

import numpy as np
import torch
import torch.nn as nn

def custom_batch_norm1d(input_tensor, eps):
    var = torch.var(input_tensor,0, unbiased = False)
    mn = torch.mean(input_tensor,0)

    normed_tensor=(input_tensor - mn)/torch.sqrt((var+eps))
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)



In [65]:
import numpy as np
all_correct = True
for eps_power in range(10):
    eps = np.power(10., -eps_power)
    batch_norm.eps = eps
    batch_norm_out = batch_norm(input_tensor)
    custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)


all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
print(all_correct)

True


In [67]:
#6.3.2
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    var = torch.var(input_tensor,0, unbiased = False)
    mn = torch.mean(input_tensor,0)

    gamma = weight
    betta = bias
    
    normed_tensor=((input_tensor - mn)/torch.sqrt((var+eps)))*gamma+betta
   
    return normed_tensor


In [68]:
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm_out = batch_norm(input_tensor)
custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)

True


In [191]:
#6.3.3
import torch
import torch.nn as nn

input_size = 3
batch_size = 5
eps = 1e-1

#Расчёт дисперсии ведётся по смещенной выборке (std)
#Расчёт скользящего среднего по несмещенной
class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight #gamma
        self.bias = bias     #betta
        self.eps = eps
        self.momentum = momentum
        self.training = False
        #дисперсия 
        self.var = 0
        self.running_var = 1
        #среднее 
        self.mn = 0
        self.running_mean = 0
        
    def __call__(self, x):#x - input tenzor
        normed_tensor = x
        #если вызван .eval
        if(self.training == True):
            
       
            
            #расчет скользящего среднего
            self.mn = torch.mean(x,0)
            self.running_mean = (1 - self.momentum)*self.running_mean + self.momentum * self.mn 
         
            #расчет дисперсии 
            self.var = torch.var(x,0, unbiased = False)
            self.running_var =  (1 - self.momentum) * self.running_mean + self.mn * momentum
            
            
           # self.var = torch.var(x,0, unbiased = True)
           # self.running_var = self.momentum * self.running_var + (1 - self.momentum) * self.var
           # self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * self.mn
            
                   
            self.training = False
            
        else:
            tmp =(x - self.running_mean)/(torch.sqrt(self.running_var + self.eps))
            normed_tensor = tmp *self.weight + self.bias
            
        return normed_tensor
    def eval(self):
        self.training = True

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)


In [192]:
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape
print('#MEANS')
print(batch_norm.running_mean)
print(custom_batch_norm1d.running_mean)
print('\n#VARS')
print(batch_norm.running_var)
print(custom_batch_norm1d.running_var)
print('\n#OUTPUTS')
print(norm_output)
print(custom_output)
print('\n#RESULT')
print(all_correct)

TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not float

In [194]:
import math

print('w4',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*math.tanh(math.tanh(math.tanh(100))))
print('w3',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1-math.tanh(math.tanh(math.tanh(100)))**2)*math.tanh(math.tanh(100)))
print('w2',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1-math.tanh(math.tanh(math.tanh(100)))**2)*(1-math.tanh(math.tanh(100))**2)*math.tanh(100))
print('w1',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1 - math.tanh(math.tanh(math.tanh(100)))**2)*(1 - math.tanh(math.tanh(100))**2)*(1 - math.tanh(100)**2)*100)

w4 0.43614538244454387
w3 0.30412468309754653
w2 0.1677068587693903
w1 0.0
